In [ ]:
# default_exp base_model
# default_cls_lvl 2

In [ ]:
%load_ext autotime
%load_ext autoreload
%autoreload 2

# Base model
> Dieses Modul stellt die Wesentlichen Funktionen für Training, Prediction und Evaluation bereit. 

Nachdem wir nun die Daten nach Train und Testset aufgeteilt haben, geht es darum das Grundmodel zu definieren. Dieses werden wir wiederum Trainieren und evaluieren. Dazu werden wir auch die offizielle Formel für die Evaluation hernehmen und zwar Mean Average Precision. Der einzige Unterschied, wir evaluieren 


In [ ]:
#export
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import cross_validate
#from community_learning.features import target_cols
from fastscript import *
from tqdm import tqdm
from itertools import compress

time: 293 ms


In [ ]:
#export
def load_data(path_train='data/interim/03_train.csv',
              path_test='data/interim/03_test.csv'):
    """load data"""
    train = pd.read_csv(path_train)
    test = pd.read_csv(path_test)
    return (train, test)    


time: 17.1 ms


In [ ]:
train_org, test = load_data()

time: 2.29 s


## Train - Test Split

In [ ]:
#export
def get_shift_cols(columns:list):
    """get a list of columns"""
    return [ col for col in columns if col[-2:] == '_s']

time: 16.8 ms


In [ ]:
shift_cols = get_shift_cols(train.columns)
assert len(shift_cols) > 0

time: 18.4 ms


In [ ]:
#export
feature_cols = ['ind_empleado', 'sexo', 'age', 'renta', 'ind_nuevo', 
                'indrel', 'indrel_1mes', 'tiprel_1mes', 'indresi', 'indext', 
                'conyuemp', 'indfall', 'tipodom', 'ind_actividad_cliente', 
                'segmento', 'antiguedad', 'pais_residencia', 'canal_entrada',
                'ind_cco_fin_ult1_s', 'ind_cder_fin_ult1_s', 'ind_cno_fin_ult1_s',
                'ind_ctju_fin_ult1_s', 'ind_ctma_fin_ult1_s', 'ind_ctop_fin_ult1_s',
                'ind_ctpp_fin_ult1_s', 'ind_deco_fin_ult1_s', 'ind_deme_fin_ult1_s',
                'ind_dela_fin_ult1_s', 'ind_ecue_fin_ult1_s', 'ind_fond_fin_ult1_s',
                'ind_hip_fin_ult1_s', 'ind_plan_fin_ult1_s', 'ind_pres_fin_ult1_s',
                'ind_reca_fin_ult1_s', 'ind_tjcr_fin_ult1_s', 'ind_valo_fin_ult1_s',
                'ind_viv_fin_ult1_s', 'ind_nomina_ult1_s', 'ind_nom_pens_ult1_s',
                'ind_recibo_ult1_s']

target_cols = ['ind_ahor_fin_ult1','ind_aval_fin_ult1','ind_cco_fin_ult1',
               'ind_cder_fin_ult1','ind_cno_fin_ult1','ind_ctju_fin_ult1',
               'ind_ctma_fin_ult1','ind_ctop_fin_ult1','ind_ctpp_fin_ult1',
               'ind_deco_fin_ult1','ind_deme_fin_ult1','ind_dela_fin_ult1',
               'ind_ecue_fin_ult1','ind_fond_fin_ult1','ind_hip_fin_ult1',
               'ind_plan_fin_ult1','ind_pres_fin_ult1','ind_reca_fin_ult1',
               'ind_tjcr_fin_ult1','ind_valo_fin_ult1','ind_viv_fin_ult1',
               'ind_nomina_ult1','ind_nom_pens_ult1','ind_recibo_ult1']

#feature_cols += get_shift_cols(train.columns) #add the shifted products as feature columns



time: 10.5 ms


In [ ]:
#export 
def get_product_dict(df:pd.DataFrame):
    """returns product_name: integer pairs"""
    products = sorted(list(train['y'].unique()))
    return { product : i for i, product in enumerate(products) }

def get_product_reverse_dict(df:pd.DataFrame):
    """returns product_name: integer pairs"""
    products = list(train['y'].unique())
    return { i : product for i, product in enumerate(products) }
    

time: 17.1 ms


In [ ]:
product_dict = get_product_dict(train)
product_reverse_dict = get_product_reverse_dict(train)
assert type(product_dict) == dict
assert type(product_reverse_dict) == dict
product_dict

{'ind_tjcr_fin_ult1': 0,
 'ind_cco_fin_ult1': 1,
 'ind_dela_fin_ult1': 2,
 'ind_cno_fin_ult1': 3,
 'ind_ctop_fin_ult1': 4,
 'ind_ecue_fin_ult1': 5,
 'ind_valo_fin_ult1': 6,
 'ind_nom_pens_ult1': 7,
 'ind_fond_fin_ult1': 8,
 'ind_deme_fin_ult1': 9,
 'ind_nomina_ult1': 10,
 'ind_reca_fin_ult1': 11,
 'ind_recibo_ult1': 12,
 'ind_ctpp_fin_ult1': 13,
 'ind_ctma_fin_ult1': 14,
 'ind_deco_fin_ult1': 15,
 'ind_hip_fin_ult1': 16,
 'ind_plan_fin_ult1': 17,
 'ind_cder_fin_ult1': 18,
 'ind_pres_fin_ult1': 19,
 'ind_viv_fin_ult1': 20,
 'ind_ctju_fin_ult1': 21}

time: 53.9 ms


In [ ]:
get_product_reverse_dict(train_org)

{0: 'ind_tjcr_fin_ult1',
 1: 'ind_cco_fin_ult1',
 2: 'ind_dela_fin_ult1',
 3: 'ind_cno_fin_ult1',
 4: 'ind_ctop_fin_ult1',
 5: 'ind_ecue_fin_ult1',
 6: 'ind_valo_fin_ult1',
 7: 'ind_nom_pens_ult1',
 8: 'ind_fond_fin_ult1',
 9: 'ind_deme_fin_ult1',
 10: 'ind_nomina_ult1',
 11: 'ind_reca_fin_ult1',
 12: 'ind_recibo_ult1',
 13: 'ind_ctpp_fin_ult1',
 14: 'ind_ctma_fin_ult1',
 15: 'ind_deco_fin_ult1',
 16: 'ind_hip_fin_ult1',
 17: 'ind_plan_fin_ult1',
 18: 'ind_cder_fin_ult1',
 19: 'ind_pres_fin_ult1',
 20: 'ind_viv_fin_ult1',
 21: 'ind_ctju_fin_ult1'}

time: 37.8 ms


In [ ]:
#export 
def encode_products(df:pd.DataFrame):
    """encode products with integer"""
    product_dict = get_product_dict(df)
    df['y'] = df['y'].map(lambda x: product_dict[x]).astype(np.int8)
    return df

time: 19 ms


In [ ]:
train = encode_products(train_org)
assert train['y'].dtype == np.int8

time: 124 ms


In [ ]:
#export
def x_y_split(df:pd.DataFrame):
    """returns 2 dataframes for X and Y variables"""
    X = df.drop('y', axis=1)
    y = df['y']
    return (X, y)

time: 19.8 ms


In [ ]:
train_X, train_y = x_y_split(train)
assert 'y' not in train_X.columns

time: 26.8 ms


## XGB train

In [ ]:
#export
def runXGB(train_X, train_y, feature_cols, seed_val=0):
    param = {}
    param['objective'] = 'multi:softprob'
    param['eta'] = 0.05
    param['max_depth'] = 8
    param['silent'] = 1
    param['num_class'] = 22
    param['eval_metric'] = "mlogloss"
    param['min_child_weight'] = 1
    param['subsample'] = 0.7
    param['colsample_bytree'] = 0.7
    param['seed'] = seed_val
    num_rounds = 50

    plst = list(param.items())
    xgtrain = xgb.DMatrix(train_X[feature_cols], label=train_y)
    model = xgb.train(plst, xgtrain, num_rounds)   
    return model
    

time: 16 ms


In [ ]:
model = runXGB(train_X, train_y, feature_cols)
assert type(model) == xgb.core.Booster

time: 17min 46s


## Evaluate

Für die [Evaluierung](https://www.kaggle.com/c/santander-product-recommendation/overview/evaluation) wird der Mean Average Precision @ 7 (MAP@7) hergenommen. Die unten stehenden Formel haben wir uns von [jturkewitz](https://github.com/jturkewitz/SideProjects/blob/4c437b02d5e017636c84cc22eb3ff71f8eea1308/Kaggle/Santander_Prod/santander_prod.py#L272) ausgeliehen. [Hier](http://sdsawtelle.github.io/blog/output/mean-average-precision-MAP-for-recommender-systems.html) noch eine gute Erklärung. 

$$
MAP@7 =  \dfrac{1} {\vert U \vert} \sum^{\vert U \vert}_{u=1} \dfrac {1} {min(m,7)} \sum^{min(n,7)}_{k=1} P(k)
$$

In [ ]:
#export
def apk(actual, predicted, k=7):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)


time: 17.7 ms


In [ ]:
xgtest = xgb.DMatrix(test[feature_cols])
preds = model.predict(xgtest)
preds = np.argsort(preds, axis=1)
preds = np.fliplr(preds)[:,:7]
preds = pd.DataFrame(preds)
preds = preds.applymap(lambda x: product_reverse_dict[x])
preds['added_products'] = preds.apply(lambda x: list(x.values), axis=1)
preds = preds['added_products']
preds

0         [ind_recibo_ult1, ind_tjcr_fin_ult1, ind_dela_...
1         [ind_cco_fin_ult1, ind_dela_fin_ult1, ind_reca...
2         [ind_nom_pens_ult1, ind_nomina_ult1, ind_cno_f...
3         [ind_cco_fin_ult1, ind_tjcr_fin_ult1, ind_reci...
4         [ind_cno_fin_ult1, ind_valo_fin_ult1, ind_fond...
                                ...                        
702430    [ind_nom_pens_ult1, ind_nomina_ult1, ind_recib...
702431    [ind_recibo_ult1, ind_nom_pens_ult1, ind_nomin...
702432    [ind_recibo_ult1, ind_nom_pens_ult1, ind_nomin...
702433    [ind_cco_fin_ult1, ind_recibo_ult1, ind_ecue_f...
702434    [ind_recibo_ult1, ind_cno_fin_ult1, ind_nom_pe...
Name: added_products, Length: 702435, dtype: object

time: 39.4 s


In [ ]:
pd.set_option('display.max_columns', None)
test['added_products'] = preds
test['truth_list'] = test[target_cols].apply(lambda x: list(compress(target_cols, x.values)), axis=1)
test[['added_products', 'truth_list']]

In [ ]:
test['apk'] = test.apply(lambda x: apk(x['truth_list'], x['added_products']),axis=1)
print(f"mean average precision = {test['apk'].mean()}")

mean average precision = 0.023004870359693223
time: 15.5 s


In [ ]:
test[['added_products', 'truth_list']]

,added_products,truth_list
0,"[ind_recibo_ult1, ind_tjcr_fin_ult1, ind_dela_...",[ind_tjcr_fin_ult1]
1,"[ind_cco_fin_ult1, ind_dela_fin_ult1, ind_reca...",[]
2,"[ind_nom_pens_ult1, ind_nomina_ult1, ind_cno_f...",[]
3,"[ind_cco_fin_ult1, ind_tjcr_fin_ult1, ind_reci...",[]
4,"[ind_cno_fin_ult1, ind_valo_fin_ult1, ind_fond...",[]
...,...,...
702430,"[ind_nom_pens_ult1, ind_nomina_ult1, ind_recib...",[]
702431,"[ind_recibo_ult1, ind_nom_pens_ult1, ind_nomin...",[]
702432,"[ind_recibo_ult1, ind_nom_pens_ult1, ind_nomin...",[]
702433,"[ind_cco_fin_ult1, ind_recibo_ult1, ind_ecue_f...",[]


time: 37.1 ms


In [ ]:
#export 
def get_results(test_data:pd.DataFrame, model:xgb.core.Booster,  target_cols:list=target_cols):
    """"""
    xgtest = xgb.DMatrix(test[feature_cols])
    preds = model.predict(xgtest)
    preds = np.argsort(preds, axis=1)
    preds = np.fliplr(preds)[:,:7]
    preds = pd.DataFrame(preds)
    preds = preds.applymap(lambda x: product_reverse_dict[x])
    preds['added_products'] = preds.apply(lambda x: list(x.values), axis=1)
    preds = preds['added_products']
    
    
    test_data['added_products'] = preds
    test_data['truth_list'] = test_data[target_cols].apply(lambda x: list(compress(target_cols, x.values)), axis=1)
    test_data[['added_products', 'truth_list']]
    test['apk'] = test.apply(lambda x: apk(x['truth_list'], x['added_products']),axis=1)
    print(f"mean average precision = {test['apk'].mean()}")
    return test_data[['id', 'added_products', 'truth_list', 'apk']]

time: 10.9 ms


In [ ]:
results = get_results(test, model, target_cols)
assert 'id' in results
assert 'truth_list' in results
assert 'apk' in results

mean average precision = 0.023004870359693223


,id,added_products,truth_list,apk
0,15889,"[ind_recibo_ult1, ind_tjcr_fin_ult1, ind_dela_...",[ind_tjcr_fin_ult1],0.5
1,15890,"[ind_cco_fin_ult1, ind_dela_fin_ult1, ind_reca...",[],0.0
2,15892,"[ind_nom_pens_ult1, ind_nomina_ult1, ind_cno_f...",[],0.0
3,15893,"[ind_cco_fin_ult1, ind_tjcr_fin_ult1, ind_reci...",[],0.0
4,15894,"[ind_cno_fin_ult1, ind_valo_fin_ult1, ind_fond...",[],0.0
...,...,...,...,...
702430,1454615,"[ind_nom_pens_ult1, ind_nomina_ult1, ind_recib...",[],0.0
702431,1454616,"[ind_recibo_ult1, ind_nom_pens_ult1, ind_nomin...",[],0.0
702432,1454617,"[ind_recibo_ult1, ind_nom_pens_ult1, ind_nomin...",[],0.0
702433,1454618,"[ind_cco_fin_ult1, ind_recibo_ult1, ind_ecue_f...",[],0.0


time: 58.4 s


## Main function

In [ ]:
#export
@call_parse
def get_base_model_results(source_train:Param("source csv file for train", str)='data/interim/03_train.csv',
                           source_test:Param("source csv file for test", str)='data/interim/03_test.csv',
                           dest:Param("destination csv file for the results", str)='data/results/base_model.csv',
                           feature_cols:Param("list of features to use for training", str)=feature_cols):
    """"""
    train_org, test = load_data(source_train, source_test)
    
    product_dict = get_product_dict(train_org)
    product_reverse_dict = get_product_reverse_dict(train_org)    
    
    train = encode_products(train_org)
    
    train_X, train_y = x_y_split(train)
    
    model = runXGB(train_X, train_y, feature_cols)
    
    results = get_results(test, model)
    
    results.to_csv(dest, index=False)
    
    return results
    

time: 22.1 ms


In [ ]:
#slow
result1 = get_base_model_results(source_train='data/interim/03_train.csv',
                                source_test='data/interim/03_test.csv',
                                dest='data/results/04_shift12.csv')
result12 = get_base_model_results(source_train='data/interim/03_train_shift12.csv',
                                source_test='data/interim/03_test_shift12.csv',
                                dest='data/results/04_shift12.csv')


## Zusammenfassung

Wie wir sehen ist unser initiales Resultat soweit ok. Leider können wir diese Zahl nicht direkt mit derjenigen von der Kaggle Competition vergleichen, da wir eine andere Periode vorhersagen. Zudem haben wir viele mögliche Features aussen vor gelassen. In einem echten Szenario würde wir vor allem noch mehr Datasets generieren und mit verschiedenen Lags und Features testen. Als Baseline für dieses Experiment ist das Model in Ordnung.

In [ ]:
from nbdev.export import *
notebook2script()

Converted 01_data_preprocess.ipynb.
Converted 02_data_Cleaning.ipynb.
Converted 03_features.ipynb.
Converted 04_base_model.ipynb.
Converted index.ipynb.
time: 123 ms


In [ ]:
#!python community_learning/base_model.py